References:
https://drive.google.com/drive/folders/1y6JWx2CpsOuka00uePe72jNgr7F9sK45?usp=sharingOriginal 

In [ ]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers,optimizers
from keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import pickle              # For saving models to disk and loading saved models
import os
import zipfile
import PIL
import PIL.Image
import pathlib
import cv2 as cv
import shutil

#1.0 YOLO V5 - Preparing the Dataset

## Google Drive 

Mounting,
Creating directories in local VM.
Setting Path Variables. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


The goal is to copy the images dataset into the dataset present in the drive.

In [ ]:
dp = '/content/drive/MyDrive/glaiml/Dataset';
dp_train = os.path.join(dp,'Car Images/Train Images')
dp_test = os.path.join(dp,'Car Images/Test Images')
os.chdir(dp)

In [ ]:
%ls

 Annotations/  'Car Images'/  'Car names and make.csv'   __MACOSX/


In [ ]:
out_dp = '/content/drive/MyDrive/glaiml/car_data_yolo'

In [ ]:
# Required only on the first run!
# Creating the directory sturcture in local VMs as per YOLO structure. 
os.chdir(out_dp)
%mkdir -p train/images test/images val/images train/labels test/labels val/labels

## DataFrame - reading from CSVs.

In [ ]:
car_img_path = os.path.join(dp,'Car Images')
car_ann_path = os.path.join(dp,'Annotations')
df_train = pd.read_csv(os.path.join(car_ann_path,'Train Annotations.csv'),names=['Image_Name','Start_x','Start_y','End_x','End_y','Image_class'])
df_test = pd.read_csv(os.path.join(car_ann_path,'Test Annotation.csv'),names=['Image_Name','Start_x','Start_y','End_x','End_y','Image_class'])
df_cars_name = pd.read_csv(os.path.join(dp,'Car names and make.csv'),names=['Car Name'])
df_train.drop([df_train.index[0]],inplace=True)
df_test.drop([df_test.index[0]],inplace=True)

In [ ]:
#df_cars_name["Car Name"].values.tolist()
df_cars_name.size

196

## Data PreProcessing

  1. Moving the images to the test directory
  1. Moving all the images to the train directory
  1. Creating a txt file corresponding to all the images
  1. Converting the images to the YOLO specific format.

In [ ]:
dp_test

'/content/drive/MyDrive/glaiml/Dataset/Car Images/Test Images'

In [ ]:
#Moving the Test Images and Train Images to the the VM's folder. Will take quite a bit of time. 

for root, dirs, files in os.walk(dp_train):
  for file in files:
    path_file = os.path.join(root,file)
    shutil.copy2(path_file,out_dp + '/train/images')

for root, dirs, files in os.walk(dp_test):
  for file in files:
    path_file = os.path.join(root,file)
    shutil.copy2(path_file,out_dp + '/test/images')

In [ ]:
# YOLO requires the normalized height and width format.
def YOLO_xywh(xmin, ymin, xmax, ymax, imgw,imgh):
  xmin = float(xmin)
  ymin = float(ymin)
  xmax = float(xmax)
  ymax = float(ymax)
  imgw = float(imgw)
  imgh = float(imgh)
  # print (float(xmin), float(ymin), float(xmax), float(ymax), float(imgw),float(imgh))
  xcen = float((xmin + xmax)) / 2 / imgw
  ycen = float((ymin + ymax)) / 2 / imgh
  w = float((xmax - xmin)) / imgw
  h = float((ymax - ymin)) / imgh
  return xcen, ycen, w, h

#Reference https://github.com/tzutalin/labelImg/blob/master/libs/yolo_io.py


In [ ]:
df_train['Image_Path'] = df_train['Image_Name'].apply(lambda x:os.path.join(out_dp + '/train/images',x))
df_test['Image_Path'] = df_test['Image_Name'].apply(lambda x:os.path.join(out_dp + '/test/images',x))


In [ ]:
df_train['Label_Path'] = df_train['Image_Name'].apply(lambda x:os.path.join(out_dp + '/train/labels',x.replace("jpg", "txt")))
df_test['Label_Path'] = df_test['Image_Name'].apply(lambda x:os.path.join(out_dp + '/test/labels',x.replace("jpg", "txt")))

The logic for regenerating the labelfiles: 
1. Iterate the df and run the image path exists loop, and then populate df with appropriate image paths and label paths
2. Then run the get image dimension for training
3. Run the iterrows for training.

In [ ]:
#df_test_merge

,Image_Name,Start_x,Start_y,End_x,End_y,Image_class,Image_Path,Label_Path,source_image_path,image_name,Orig_img_height,Orig_img_width
0,00001.jpg,30,52.0,246.0,147.0,181,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/Dataset/Car Imag...,00001.jpg,182,276
1,00002.jpg,100,19.0,576.0,203.0,103,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/Dataset/Car Imag...,00002.jpg,360,640
2,00003.jpg,51,105.0,968.0,659.0,145,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/Dataset/Car Imag...,00003.jpg,741,1024
3,00004.jpg,67,84.0,581.0,407.0,187,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/Dataset/Car Imag...,00004.jpg,480,640
4,00005.jpg,140,151.0,593.0,339.0,185,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/Dataset/Car Imag...,00005.jpg,373,600
...,...,...,...,...,...,...,...,...,...,...,...,...
8036,08037.jpg,49,57.0,1169.0,669.0,63,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/Dataset/Car Imag...,08037.jpg,800,1200
8037,08038.jpg,23,18.0,640.0,459.0,16,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/Dataset/Car Imag...,08038.jpg,480,640
8038,08039.jpg,33,27.0,602.0,252.0,17,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/Dataset/Car Imag...,08039.jpg,303,635
8039,08040.jpg,33,142.0,521.0,376.0,38,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/car_data_yolo/te...,/content/drive/MyDrive/glaiml/Dataset/Car Imag...,08040.jpg,427,640


In [ ]:
# df_test_merge[['Orig_img_height']]= df_test_merge.apply(lambda _: 0, axis=1)
# df_test_merge[['Orig_img_width']]= df_test_merge.apply(lambda _: 0, axis=1)

In [ ]:
# +++++++
# DO NOT RUN THIS
# +++++
"""
for index, row in df_train.iterrows():
  if not os.path.exists(row['Image_Path']):
    edit_row = df_train.iloc[[index]]
    df_train.at[index, 'Label_Path'] = row['Label_Path'].replace('train','val')
    df_train.at[index, 'Image_Path'] = row['Image_Path'].replace('train','val')
"""

In [ ]:
# +++++++
# DO NOT RUN THIS
# +++++

"""
# Tester Utility Function
counter = 0
for index, row in df_train.iterrows():
  # if not os.path.exists(row['Image_Path']):
  if not row['Orig_img_width']:
    counter= counter+1
print (counter)
"""

0


#### Testing 
1. The idea is to create a new dataframe with ImageName and the path by going through the pat 

In [ ]:
df_test_temp = pd.DataFrame({'source_image_path':[],'image_name':[]})

In [ ]:
df_test_merge.columns

Index(['Image_Name', 'Start_x', 'Start_y', 'End_x', 'End_y', 'Image_class',
       'Image_Path', 'Label_Path', 'source_image_path', 'image_name'],
      dtype='object')

In [ ]:
df_test_merge = pd.merge(left=df_test, right=df_test_temp, left_on='Image_Name', right_on='image_name')

In [ ]:
for root, dirs, files in os.walk(dp_test):
  for file in files:
    path_file = os.path.join(root,file)
    df_test_temp.loc[len(df_test_temp.index)] = [path_file,file]

In [ ]:
df_train.to_csv('/content/drive/MyDrive/glaiml/car_data_yolo/df_train.csv')

In [ ]:
df_test_merge.to_csv('/content/drive/MyDrive/glaiml/car_data_yolo/df_test.csv')

In [ ]:
df_test = df_test_merge

In [ ]:
for index, row in df_test_merge.iterrows():
  img = cv.imread(row['Image_Path'])
  if img is None:
    shutil.copy2(row['source_image_path'],'/content/drive/MyDrive/glaiml/car_data_yolo/test/images/')
    img = cv.imread(row['Image_Path'])
  df_test_merge.at[index, 'Orig_img_height'] = int(img.shape[0])
  df_test_merge.at[index, 'Orig_img_width'] = int(img.shape[1])


#### Others

In [ ]:
#Populating dataset with the original image height and width: 
def get_image_dims(df,col):
  height = []
  width = []
  for img_path in df[col].values:
    img = cv.imread(img_path)
    if img is not None:
      height.append(int(img.shape[0]))
      width.append(int(img.shape[1]))

  return height,width

df_train['Orig_img_height'],df_train['Orig_img_width'] =  get_image_dims(df_train,'Image_Path')
df_test['Orig_img_height'],df_test['Orig_img_width'] =  get_image_dims(df_test,'Image_Path')

In [ ]:
#Iterate over the rows
for index, row in df_train.iterrows():
    x, y, w, h = YOLO_xywh(row['Start_x'],	row['Start_y'],	row['End_x'],	row['End_y'], row['Orig_img_width'], row['Orig_img_height'])
    image_class_index = int(row['Image_class']) - 1
    with open(row['Label_Path'], "w") as text_file:
      print(f"{image_class_index} {x} {y} {w} {h}", file=text_file)

for index, row in df_test.iterrows():
    x, y, w, h = YOLO_xywh(row['Start_x'],	row['Start_y'],	row['End_x'],	row['End_y'],row['Orig_img_width'], row['Orig_img_height'])
    image_class_index = int(row['Image_class']) - 1
    with open(row['Image_Path'], "w") as text_file:
      print(f"{image_class_index} {x} {y} {w} {h}", file=text_file)

In [ ]:

#This is to regenerate the items with all the code.


#Modifying the above code with an error check!
for index, row in df_train.iterrows():
    x, y, w, h = YOLO_xywh(row['Start_x'],	row['Start_y'],	row['End_x'],	row['End_y'], row['Orig_img_width'], row['Orig_img_height'])
    with open(row['Label_Path'], "w") as text_file:
      print(f"{row['Image_class']} {x} {y} {w} {h}", file=text_file)

for index, row in df_test.iterrows():
    x, y, w, h = YOLO_xywh(row['Start_x'],	row['Start_y'],	row['End_x'],	row['End_y'],row['Orig_img_width'], row['Orig_img_height'])
    with open(row['Image_Path'], "w") as text_file:
      print(f"{row['Image_class']} {x} {y} {w} {h}", file=text_file)

## 1.3 Splitting the folders
1. Create a test directory
2. Split the train images into two validation 
3. Create its own directory and copy it 
4. Copy the label files as well.



In [ ]:
#mkdir -p train/images train/labels val/images val/labels
%pwd

'/content/ftest'

In [ ]:
#populate our experimental folder
list_int = np.arange(1,20,1).tolist()
list_str = map(str,list_int)
for i in list_str:
  os.mknod(os.path.join('train/images/',i+'.jpg'))
  os.mknod(os.path.join('train/labels/',i+'.txt'))


In [ ]:
ls /content/drive/MyDrive/glaiml/car_data_yolo/train/images | wc -l

6581


In [ ]:
#Implementing the splitting folder logic
#https://www.kaggle.com/questions-and-answers/102677 

source_train = out_dp + '/train/images/'
dest_val = out_dp + '/val/images'
allFileNames = os.listdir(source_train)
np.random.shuffle(allFileNames)
test_ratio = .2
train_FileNames, val_FileNames = np.split(np.array(allFileNames),[int(len(allFileNames)* (1 - test_ratio))])
# print (train_FileNames)
# print (val_FileNames)

for name in val_FileNames:
  # Preparing to move the corresponding label files as well.
  label_FileName = name.replace('jpg','txt')
  label_PathName = source_train.replace('images','labels')+label_FileName

  shutil.move(source_train+name,dest_val)
  shutil.move(label_PathName,dest_val.replace('images','labels'))



In [ ]:
ls train/labels

10.txt  14.txt  16.txt  18.txt  1.txt  3.txt  5.txt  9.txt
12.txt  15.txt  17.txt  19.txt  2.txt  4.txt  6.txt


# 2.0 Installing YOLOV5 

1. Download YOLOV5 from github. 
2. Install the requirements document
3. Configure the requirement yaml.

In [ ]:
%%capture
%cd /content/
!git clone https://github.com/ultralytics/yolov5.git
%cd /content/yolov5
!pip install -r requirements.txt

In [ ]:
%cd data/

/content/yolov5/data


In [ ]:
!touch carData.yaml

In [ ]:
%%writefile carData.yaml

train: /content/drive/MyDrive/glaiml/car_data_yolo/train/images
val: /content/drive/MyDrive/glaiml/car_data_yolo/val/images
test: /content/drive/MyDrive/glaiml/car_data_yolo/test/images

# number of classes
nc: 196

# class names
names: ['AM General Hummer SUV 2000', 'Acura RL Sedan 2012', 'Acura TL Sedan 2012', 'Acura TL Type-S 2008', 'Acura TSX Sedan 2012', 'Acura Integra Type R 2001', 'Acura ZDX Hatchback 2012', 'Aston Martin V8 Vantage Convertible 2012', 'Aston Martin V8 Vantage Coupe 2012', 'Aston Martin Virage Convertible 2012', 'Aston Martin Virage Coupe 2012', 'Audi RS 4 Convertible 2008', 'Audi A5 Coupe 2012', 'Audi TTS Coupe 2012', 'Audi R8 Coupe 2012', 'Audi V8 Sedan 1994', 'Audi 100 Sedan 1994', 'Audi 100 Wagon 1994', 'Audi TT Hatchback 2011', 'Audi S6 Sedan 2011', 'Audi S5 Convertible 2012', 'Audi S5 Coupe 2012', 'Audi S4 Sedan 2012', 'Audi S4 Sedan 2007', 'Audi TT RS Coupe 2012', 'BMW ActiveHybrid 5 Sedan 2012', 'BMW 1 Series Convertible 2012', 'BMW 1 Series Coupe 2012', 'BMW 3 Series Sedan 2012', 'BMW 3 Series Wagon 2012', 'BMW 6 Series Convertible 2007', 'BMW X5 SUV 2007', 'BMW X6 SUV 2012', 'BMW M3 Coupe 2012', 'BMW M5 Sedan 2010', 'BMW M6 Convertible 2010', 'BMW X3 SUV 2012', 'BMW Z4 Convertible 2012', 'Bentley Continental Supersports Conv. Convertible 2012', 'Bentley Arnage Sedan 2009', 'Bentley Mulsanne Sedan 2011', 'Bentley Continental GT Coupe 2012', 'Bentley Continental GT Coupe 2007', 'Bentley Continental Flying Spur Sedan 2007', 'Bugatti Veyron 16.4 Convertible 2009', 'Bugatti Veyron 16.4 Coupe 2009', 'Buick Regal GS 2012', 'Buick Rainier SUV 2007', 'Buick Verano Sedan 2012', 'Buick Enclave SUV 2012', 'Cadillac CTS-V Sedan 2012', 'Cadillac SRX SUV 2012', 'Cadillac Escalade EXT Crew Cab 2007', 'Chevrolet Silverado 1500 Hybrid Crew Cab 2012', 'Chevrolet Corvette Convertible 2012', 'Chevrolet Corvette ZR1 2012', 'Chevrolet Corvette Ron Fellows Edition Z06 2007', 'Chevrolet Traverse SUV 2012', 'Chevrolet Camaro Convertible 2012', 'Chevrolet HHR SS 2010', 'Chevrolet Impala Sedan 2007', 'Chevrolet Tahoe Hybrid SUV 2012', 'Chevrolet Sonic Sedan 2012', 'Chevrolet Express Cargo Van 2007', 'Chevrolet Avalanche Crew Cab 2012', 'Chevrolet Cobalt SS 2010', 'Chevrolet Malibu Hybrid Sedan 2010', 'Chevrolet TrailBlazer SS 2009', 'Chevrolet Silverado 2500HD Regular Cab 2012', 'Chevrolet Silverado 1500 Classic Extended Cab 2007', 'Chevrolet Express Van 2007', 'Chevrolet Monte Carlo Coupe 2007', 'Chevrolet Malibu Sedan 2007', 'Chevrolet Silverado 1500 Extended Cab 2012', 'Chevrolet Silverado 1500 Regular Cab 2012', 'Chrysler Aspen SUV 2009', 'Chrysler Sebring Convertible 2010', 'Chrysler Town and Country Minivan 2012', 'Chrysler 300 SRT-8 2010', 'Chrysler Crossfire Convertible 2008', 'Chrysler PT Cruiser Convertible 2008', 'Daewoo Nubira Wagon 2002', 'Dodge Caliber Wagon 2012', 'Dodge Caliber Wagon 2007', 'Dodge Caravan Minivan 1997', 'Dodge Ram Pickup 3500 Crew Cab 2010', 'Dodge Ram Pickup 3500 Quad Cab 2009', 'Dodge Sprinter Cargo Van 2009', 'Dodge Journey SUV 2012', 'Dodge Dakota Crew Cab 2010', 'Dodge Dakota Club Cab 2007', 'Dodge Magnum Wagon 2008', 'Dodge Challenger SRT8 2011', 'Dodge Durango SUV 2012', 'Dodge Durango SUV 2007', 'Dodge Charger Sedan 2012', 'Dodge Charger SRT-8 2009', 'Eagle Talon Hatchback 1998', 'FIAT 500 Abarth 2012', 'FIAT 500 Convertible 2012', 'Ferrari FF Coupe 2012', 'Ferrari California Convertible 2012', 'Ferrari 458 Italia Convertible 2012', 'Ferrari 458 Italia Coupe 2012', 'Fisker Karma Sedan 2012', 'Ford F-450 Super Duty Crew Cab 2012', 'Ford Mustang Convertible 2007', 'Ford Freestar Minivan 2007', 'Ford Expedition EL SUV 2009', 'Ford Edge SUV 2012', 'Ford Ranger SuperCab 2011', 'Ford GT Coupe 2006', 'Ford F-150 Regular Cab 2012', 'Ford F-150 Regular Cab 2007', 'Ford Focus Sedan 2007', 'Ford E-Series Wagon Van 2012', 'Ford Fiesta Sedan 2012', 'GMC Terrain SUV 2012', 'GMC Savana Van 2012', 'GMC Yukon Hybrid SUV 2012', 'GMC Acadia SUV 2012', 'GMC Canyon Extended Cab 2012', 'Geo Metro Convertible 1993', 'HUMMER H3T Crew Cab 2010', 'HUMMER H2 SUT Crew Cab 2009', 'Honda Odyssey Minivan 2012', 'Honda Odyssey Minivan 2007', 'Honda Accord Coupe 2012', 'Honda Accord Sedan 2012', 'Hyundai Veloster Hatchback 2012', 'Hyundai Santa Fe SUV 2012', 'Hyundai Tucson SUV 2012', 'Hyundai Veracruz SUV 2012', 'Hyundai Sonata Hybrid Sedan 2012', 'Hyundai Elantra Sedan 2007', 'Hyundai Accent Sedan 2012', 'Hyundai Genesis Sedan 2012', 'Hyundai Sonata Sedan 2012', 'Hyundai Elantra Touring Hatchback 2012', 'Hyundai Azera Sedan 2012', 'Infiniti G Coupe IPL 2012', 'Infiniti QX56 SUV 2011', 'Isuzu Ascender SUV 2008', 'Jaguar XK XKR 2012', 'Jeep Patriot SUV 2012', 'Jeep Wrangler SUV 2012', 'Jeep Liberty SUV 2012', 'Jeep Grand Cherokee SUV 2012', 'Jeep Compass SUV 2012', 'Lamborghini Reventon Coupe 2008', 'Lamborghini Aventador Coupe 2012', 'Lamborghini Gallardo LP 570-4 Superleggera 2012', 'Lamborghini Diablo Coupe 2001', 'Land Rover Range Rover SUV 2012', 'Land Rover LR2 SUV 2012', 'Lincoln Town Car Sedan 2011', 'MINI Cooper Roadster Convertible 2012', 'Maybach Landaulet Convertible 2012', 'Mazda Tribute SUV 2011', 'McLaren MP4-12C Coupe 2012', 'Mercedes-Benz 300-Class Convertible 1993', 'Mercedes-Benz C-Class Sedan 2012', 'Mercedes-Benz SL-Class Coupe 2009', 'Mercedes-Benz E-Class Sedan 2012', 'Mercedes-Benz S-Class Sedan 2012', 'Mercedes-Benz Sprinter Van 2012', 'Mitsubishi Lancer Sedan 2012', 'Nissan Leaf Hatchback 2012', 'Nissan NV Passenger Van 2012', 'Nissan Juke Hatchback 2012', 'Nissan 240SX Coupe 1998', 'Plymouth Neon Coupe 1999', 'Porsche Panamera Sedan 2012', 'Ram C/V Cargo Van Minivan 2012', 'Rolls-Royce Phantom Drophead Coupe Convertible 2012', 'Rolls-Royce Ghost Sedan 2012', 'Rolls-Royce Phantom Sedan 2012', 'Scion xD Hatchback 2012', 'Spyker C8 Convertible 2009', 'Spyker C8 Coupe 2009', 'Suzuki Aerio Sedan 2007', 'Suzuki Kizashi Sedan 2012', 'Suzuki SX4 Hatchback 2012', 'Suzuki SX4 Sedan 2012', 'Tesla Model S Sedan 2012', 'Toyota Sequoia SUV 2012', 'Toyota Camry Sedan 2012', 'Toyota Corolla Sedan 2012', 'Toyota 4Runner SUV 2012', 'Volkswagen Golf Hatchback 2012', 'Volkswagen Golf Hatchback 1991', 'Volkswagen Beetle Hatchback 2012', 'Volvo C30 Hatchback 2012', 'Volvo 240 Sedan 1993', 'Volvo XC90 SUV 2007', 'smart fortwo Convertible 2012']

Overwriting carData.yaml


In [ ]:
%cat carData.yaml


train: /content/drive/MyDrive/glaiml/car_data_yolo/train/images
val: /content/drive/MyDrive/glaiml/car_data_yolo/val/images
test: /content/drive/MyDrive/glaiml/car_data_yolo/test/images

# number of classes
nc: 196

# class names
names: ['AM General Hummer SUV 2000', 'Acura RL Sedan 2012', 'Acura TL Sedan 2012', 'Acura TL Type-S 2008', 'Acura TSX Sedan 2012', 'Acura Integra Type R 2001', 'Acura ZDX Hatchback 2012', 'Aston Martin V8 Vantage Convertible 2012', 'Aston Martin V8 Vantage Coupe 2012', 'Aston Martin Virage Convertible 2012', 'Aston Martin Virage Coupe 2012', 'Audi RS 4 Convertible 2008', 'Audi A5 Coupe 2012', 'Audi TTS Coupe 2012', 'Audi R8 Coupe 2012', 'Audi V8 Sedan 1994', 'Audi 100 Sedan 1994', 'Audi 100 Wagon 1994', 'Audi TT Hatchback 2011', 'Audi S6 Sedan 2011', 'Audi S5 Convertible 2012', 'Audi S5 Coupe 2012', 'Audi S4 Sedan 2012', 'Audi S4 Sedan 2007', 'Audi TT RS Coupe 2012', 'BMW ActiveHybrid 5 Sedan 2012', 'BMW 1 Series Convertible 2012', 'BMW 1 Series Coupe 2012',

In [ ]:
names = ['AM General Hummer SUV 2000', 'Acura RL Sedan 2012', 'Acura TL Sedan 2012', 'Acura TL Type-S 2008', 'Acura TSX Sedan 2012', 'Acura Integra Type R 2001', 'Acura ZDX Hatchback 2012', 'Aston Martin V8 Vantage Convertible 2012', 'Aston Martin V8 Vantage Coupe 2012', 'Aston Martin Virage Convertible 2012', 'Aston Martin Virage Coupe 2012', 'Audi RS 4 Convertible 2008', 'Audi A5 Coupe 2012', 'Audi TTS Coupe 2012', 'Audi R8 Coupe 2012', 'Audi V8 Sedan 1994', 'Audi 100 Sedan 1994', 'Audi 100 Wagon 1994', 'Audi TT Hatchback 2011', 'Audi S6 Sedan 2011', 'Audi S5 Convertible 2012', 'Audi S5 Coupe 2012', 'Audi S4 Sedan 2012', 'Audi S4 Sedan 2007', 'Audi TT RS Coupe 2012', 'BMW ActiveHybrid 5 Sedan 2012', 'BMW 1 Series Convertible 2012', 'BMW 1 Series Coupe 2012', 'BMW 3 Series Sedan 2012', 'BMW 3 Series Wagon 2012', 'BMW 6 Series Convertible 2007', 'BMW X5 SUV 2007', 'BMW X6 SUV 2012', 'BMW M3 Coupe 2012', 'BMW M5 Sedan 2010', 'BMW M6 Convertible 2010', 'BMW X3 SUV 2012', 'BMW Z4 Convertible 2012', 'Bentley Continental Supersports Conv. Convertible 2012', 'Bentley Arnage Sedan 2009', 'Bentley Mulsanne Sedan 2011', 'Bentley Continental GT Coupe 2012', 'Bentley Continental GT Coupe 2007', 'Bentley Continental Flying Spur Sedan 2007', 'Bugatti Veyron 16.4 Convertible 2009', 'Bugatti Veyron 16.4 Coupe 2009', 'Buick Regal GS 2012', 'Buick Rainier SUV 2007', 'Buick Verano Sedan 2012', 'Buick Enclave SUV 2012', 'Cadillac CTS-V Sedan 2012', 'Cadillac SRX SUV 2012', 'Cadillac Escalade EXT Crew Cab 2007', 'Chevrolet Silverado 1500 Hybrid Crew Cab 2012', 'Chevrolet Corvette Convertible 2012', 'Chevrolet Corvette ZR1 2012', 'Chevrolet Corvette Ron Fellows Edition Z06 2007', 'Chevrolet Traverse SUV 2012', 'Chevrolet Camaro Convertible 2012', 'Chevrolet HHR SS 2010', 'Chevrolet Impala Sedan 2007', 'Chevrolet Tahoe Hybrid SUV 2012', 'Chevrolet Sonic Sedan 2012', 'Chevrolet Express Cargo Van 2007', 'Chevrolet Avalanche Crew Cab 2012', 'Chevrolet Cobalt SS 2010', 'Chevrolet Malibu Hybrid Sedan 2010', 'Chevrolet TrailBlazer SS 2009', 'Chevrolet Silverado 2500HD Regular Cab 2012', 'Chevrolet Silverado 1500 Classic Extended Cab 2007', 'Chevrolet Express Van 2007', 'Chevrolet Monte Carlo Coupe 2007', 'Chevrolet Malibu Sedan 2007', 'Chevrolet Silverado 1500 Extended Cab 2012', 'Chevrolet Silverado 1500 Regular Cab 2012', 'Chrysler Aspen SUV 2009', 'Chrysler Sebring Convertible 2010', 'Chrysler Town and Country Minivan 2012', 'Chrysler 300 SRT-8 2010', 'Chrysler Crossfire Convertible 2008', 'Chrysler PT Cruiser Convertible 2008', 'Daewoo Nubira Wagon 2002', 'Dodge Caliber Wagon 2012', 'Dodge Caliber Wagon 2007', 'Dodge Caravan Minivan 1997', 'Dodge Ram Pickup 3500 Crew Cab 2010', 'Dodge Ram Pickup 3500 Quad Cab 2009', 'Dodge Sprinter Cargo Van 2009', 'Dodge Journey SUV 2012', 'Dodge Dakota Crew Cab 2010', 'Dodge Dakota Club Cab 2007', 'Dodge Magnum Wagon 2008', 'Dodge Challenger SRT8 2011', 'Dodge Durango SUV 2012', 'Dodge Durango SUV 2007', 'Dodge Charger Sedan 2012', 'Dodge Charger SRT-8 2009', 'Eagle Talon Hatchback 1998', 'FIAT 500 Abarth 2012', 'FIAT 500 Convertible 2012', 'Ferrari FF Coupe 2012', 'Ferrari California Convertible 2012', 'Ferrari 458 Italia Convertible 2012', 'Ferrari 458 Italia Coupe 2012', 'Fisker Karma Sedan 2012', 'Ford F-450 Super Duty Crew Cab 2012', 'Ford Mustang Convertible 2007', 'Ford Freestar Minivan 2007', 'Ford Expedition EL SUV 2009', 'Ford Edge SUV 2012', 'Ford Ranger SuperCab 2011', 'Ford GT Coupe 2006', 'Ford F-150 Regular Cab 2012', 'Ford F-150 Regular Cab 2007', 'Ford Focus Sedan 2007', 'Ford E-Series Wagon Van 2012', 'Ford Fiesta Sedan 2012', 'GMC Terrain SUV 2012', 'GMC Savana Van 2012', 'GMC Yukon Hybrid SUV 2012', 'GMC Acadia SUV 2012', 'GMC Canyon Extended Cab 2012', 'Geo Metro Convertible 1993', 'HUMMER H3T Crew Cab 2010', 'HUMMER H2 SUT Crew Cab 2009', 'Honda Odyssey Minivan 2012', 'Honda Odyssey Minivan 2007', 'Honda Accord Coupe 2012', 'Honda Accord Sedan 2012', 'Hyundai Veloster Hatchback 2012', 'Hyundai Santa Fe SUV 2012', 'Hyundai Tucson SUV 2012', 'Hyundai Veracruz SUV 2012', 'Hyundai Sonata Hybrid Sedan 2012', 'Hyundai Elantra Sedan 2007', 'Hyundai Accent Sedan 2012', 'Hyundai Genesis Sedan 2012', 'Hyundai Sonata Sedan 2012', 'Hyundai Elantra Touring Hatchback 2012', 'Hyundai Azera Sedan 2012', 'Infiniti G Coupe IPL 2012', 'Infiniti QX56 SUV 2011', 'Isuzu Ascender SUV 2008', 'Jaguar XK XKR 2012', 'Jeep Patriot SUV 2012', 'Jeep Wrangler SUV 2012', 'Jeep Liberty SUV 2012', 'Jeep Grand Cherokee SUV 2012', 'Jeep Compass SUV 2012', 'Lamborghini Reventon Coupe 2008', 'Lamborghini Aventador Coupe 2012', 'Lamborghini Gallardo LP 570-4 Superleggera 2012', 'Lamborghini Diablo Coupe 2001', 'Land Rover Range Rover SUV 2012', 'Land Rover LR2 SUV 2012', 'Lincoln Town Car Sedan 2011', 'MINI Cooper Roadster Convertible 2012', 'Maybach Landaulet Convertible 2012', 'Mazda Tribute SUV 2011', 'McLaren MP4-12C Coupe 2012', 'Mercedes-Benz 300-Class Convertible 1993', 'Mercedes-Benz C-Class Sedan 2012', 'Mercedes-Benz SL-Class Coupe 2009', 'Mercedes-Benz E-Class Sedan 2012', 'Mercedes-Benz S-Class Sedan 2012', 'Mercedes-Benz Sprinter Van 2012', 'Mitsubishi Lancer Sedan 2012', 'Nissan Leaf Hatchback 2012', 'Nissan NV Passenger Van 2012', 'Nissan Juke Hatchback 2012', 'Nissan 240SX Coupe 1998', 'Plymouth Neon Coupe 1999', 'Porsche Panamera Sedan 2012', 'Ram C/V Cargo Van Minivan 2012', 'Rolls-Royce Phantom Drophead Coupe Convertible 2012', 'Rolls-Royce Ghost Sedan 2012', 'Rolls-Royce Phantom Sedan 2012', 'Scion xD Hatchback 2012', 'Spyker C8 Convertible 2009', 'Spyker C8 Coupe 2009', 'Suzuki Aerio Sedan 2007', 'Suzuki Kizashi Sedan 2012', 'Suzuki SX4 Hatchback 2012', 'Suzuki SX4 Sedan 2012', 'Tesla Model S Sedan 2012', 'Toyota Sequoia SUV 2012', 'Toyota Camry Sedan 2012', 'Toyota Corolla Sedan 2012', 'Toyota 4Runner SUV 2012', 'Volkswagen Golf Hatchback 2012', 'Volkswagen Golf Hatchback 1991', 'Volkswagen Beetle Hatchback 2012', 'Volvo C30 Hatchback 2012', 'Volvo 240 Sedan 1993', 'Volvo XC90 SUV 2007', 'smart fortwo Convertible 2012']
len(names)

196

#3.0 YOLO V5 - Training / Testing the Dataset

In [ ]:
%cd /content/yolov5

/content/yolov5


In [ ]:
!python train.py --img 160 --cfg yolov5s.yaml --hyp hyp.scratch.yaml --batch 32 --epochs 100 --data carData.yaml --weights yolov5s.pt --workers 24 --name yolo_car_classification

train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=carData.yaml, hyp=hyp.scratch.yaml, epochs=100, batch_size=32, imgsz=160, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=24, project=runs/train, name=yolo_car_classification, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-155-gdc54ed5 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, tra

In [ ]:
!python test.py --weights runs/train/yolo_car_classification/weights/best.pt --data carData.yaml --task test --name yolo_car_class_det


In [ ]:
cd /content/drive/MyDrive/glaiml/car_data_yolo/val/labels

/content/drive/MyDrive/glaiml/car_data_yolo/val/labels


In [ ]:
path = '/content/drive/MyDrive/glaiml/car_data_yolo/train/labels' #path of labels
labels = os.listdir('path')
for x in labels[:10]:
  with open('path'+x) as f:
    lines = f.read().splitlines()
    for y in lines:
      if y[:1]!='0':
          print(x)

References:
Try and have this reference later. 